# **Exploration des données - DynamicEarthNet**
## **1. Présentation du contexte et du jeu de données**

Dans ce projet, l’objectif principal est la réalisation d’une **segmentation sémantique spatiale** à partir d’images satellites issues de la base de données **DynamicEarthNet**. Cette dernière se compose d’images satellites multispectrales (RGB + proche infrarouge) annotées manuellement pour différents types de couverture terrestre.

Le jeu de données se caractérise par :
- **Résolution spatiale** : 3 mètres par pixel (chaque image fait 1024x1024 pixels).
- **Annotations manuelles précises**, pixel par pixel, classifiant la couverture terrestre en plusieurs classes sémantiques clairement définies.

> Source : *DynamicEarthNet Paper, Toker et al. 2022*.

---



La première étape de notre projet à donc été de visualiser et de prendre en main la données. Pour cela deux module ont été crée : [sat_image_reader](src/sat_image_reader.py) et [classes_reader](src/classes_reader.py)


## **2. Classes sémantiques utilisées dans l'exploration**

Le jeu de données DynamicEarthNet comporte au total **7 classes sémantiques** différentes :

1. **Impervious Surface** – Surfaces imperméables (routes, bâtiments).
2. **Forest & Other Vegetation** – Forêt et autres végétations.
3. **Soil** – Sols nus ou terres agricoles non cultivées.
4. **Agriculture** – Zones agricoles cultivées.
5. **Wetlands** – Zones humides.
6. **Water** – Plans d’eau.
7. **Snow & Ice** – Neige et glace (rare, non incluse dans cette analyse).

Dans la zone étudiée (**10N-123W-45N**), seules **3 classes principales** sont présentes :

- **Impervious Surface (bande 1)**
- **Forest and Other Vegetation (bande 3)**
- **Soil (bande 5)**

---



## **3. Approche méthodologique d’exploration**

### **a. Exploration initiale des fichiers raster (.tif)**

L’analyse initiale a consisté à examiner les fichiers raster pour :
- Lire les métadonnées (dimensions, nombre de bandes, résolution).
- Visualiser les bandes individuellement :
  - Classification binaire (valeurs 0 ou 255).
  - Identification des pixels appartenant simultanément à plusieurs classes.
- Générer des histogrammes confirmant la distribution binaire des pixels.

### **b. Validation spatiale avec les fichiers vectoriels (.geojson)**

Les fichiers **GeoJSON** ont été utilisés pour valider spatialement les annotations :
- Chargement et visualisation des contours géographiques de chaque classe.
- Superposition des géométries vectorielles sur les images raster.
- Confirmation de la cohérence et de la précision des annotations.

---



## **4. Analyse complémentaire : NDVI**

Un calcul de l’**indice de végétation NDVI** a été effectué à partir des bandes rouge et proche infrarouge :

- **Objectif** : Vérifier la correspondance entre le NDVI et les annotations de végétation.
- **Résultats** :
  - NDVI élevé (> 0.5) → correspondance forte avec la classe **Forest & Other Vegetation**.
  - Confirmation de la pertinence du NDVI comme attribut supplémentaire pour améliorer la segmentation.

---



## **5. Analyse de la qualité des données (PF-QA)**

L’étude des fichiers **PF-QA (Planet Fusion Quality Assurance)** a permis de :
- Identifier les pixels basés sur des observations satellites directes vs interpolées.
- Vérifier la cohérence spatiale des annotations pour garantir la fiabilité des données d’apprentissage.

---



## **6. Synthèse des résultats de l’exploration**

L’exploration des données DynamicEarthNet a permis de dégager les éléments suivants :
- **Trois classes principales retenues** : Impervious Surface, Forest & Other Vegetation, et Soil.
- Les annotations raster sont **clairement binaires**, facilitant l’apprentissage supervisé.
- Validation rigoureuse des annotations grâce aux données vectorielles.
- **Le NDVI est un bon indicateur** pour compléter la segmentation des zones végétalisées.
- **Les données PF-QA doivent être prises en compte** pour éviter d’entraîner le modèle sur des pixels interpolés.

Ces observations serviront de **base pour l’entraînement d’un modèle de segmentation** précis et fiable à partir des images satellites de DynamicEarthNet.